In [1]:
PATH_TO_STYLES = "./styles"
PATH_TO_VGG_MODEL = "./models"
PATH_TO_TRAINED_MODEL = "./output"
CUDA_LEARN = False
TRAIN_DATASET = "./train_dataset"
TEST_DATASET = "./test_dataset"

In [45]:
!pip3 install telegram-send==0.34
!pip3 install python-telegram-bot==13.5
!pip install python-dotenv==1.0.0

In [11]:
import os
from dotenv import load_dotenv
from pathlib import Path

path='../config/.env'
load_dotenv(dotenv_path=path, verbose=True)

PROJECT_DIR = os.environ.get("PROJECT_ROOT", "")
TG_NOTIFY_BOT_TOKEN = os.environ.get("TG_NOTIFY_BOT_TOKEN", "")
TRAIN_STYLE_PATH = os.environ.get("TRAIN_STYLE_PATH", "./styles/mosaic.jpg")

In [5]:
!echo "$TG_NOTIFY_BOT_TOKEN"> ts.conf
!telegram-send --configure < ts.conf

Talk with the BotFather on Telegram (https://telegram.me/BotFather), create a bot and insert the token
❯ Connected with jupyter_s1sharp_bot.

Please add jupyter_s1sharp_bot on Telegram (https://telegram.me/jupyter_s1sharp_bot)
and send it the password: 32612

Congratulations s1Sharp! 
telegram-send is now ready for use!


In [33]:
!telegram-send "Operation ok"

In [16]:
import os
tg_file = "./ts.conf"
os.environ["TG_SEND_FILE"] = tg_file

In [91]:
!telegram-send --file "$TG_SEND_FILE"
!telegram-send -f {tg_file} --caption "Hello"

In [ ]:
!wget http://images.cocodataset.org/zips/train2014.zip # https://msvocds.blob.core.windows.net/coco2014/train2014.zip
!wget http://images.cocodataset.org/zips/val2014.zip # https://msvocds.blob.core.windows.net/coco2014/val2014.zip

--2023-04-08 20:59:12--  http://images.cocodataset.org/zips/train2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.225.137, 52.217.230.33, 52.217.122.25, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.225.137|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13510573713 (13G) [application/zip]
Saving to: ‘train2014.zip’

train2014.zip       100%[===================>]  12.58G  59.8MB/s    in 3m 40s  

2023-04-08 21:02:52 (58.6 MB/s) - ‘train2014.zip’ saved [13510573713/13510573713]

--2023-04-08 21:02:52--  http://images.cocodataset.org/zips/val2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.108.67, 52.217.206.233, 54.231.128.129, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.108.67|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6645013297 (6.2G) [application/zip]
Saving to: ‘val2014.zip’

val2014.zip         100%[=================

In [ ]:
!mkdir -p {TRAIN_DATASET} {TEST_DATASET}

In [ ]:
!unzip -q train2014.zip -d {TRAIN_DATASET}
!unzip -q val2014.zip -d {TEST_DATASET}

In [ ]:
!pip3 install numpy opencv_python Pillow torch torchfile torchvision tqdm av

In [ ]:
import os
import sys
import time
import numpy as np
from tqdm import tqdm, trange

import torch
from torch.optim import Adam
from torch.autograd import Variable
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

# import utils
# from net import Net, Vgg16

# from option import Options


In [ ]:
from torchvision import transforms
import torch
import numpy as np


# Mean and standard deviation used for pre-trained PyTorch models
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])


def gram_matrix(y):
    """ Returns the gram matrix of y (used to compute style loss) """
    (b, c, h, w) = y.size()
    features = y.view(b, c, w * h)
    features_t = features.transpose(1, 2)
    gram = features.bmm(features_t) / (c * h * w)
    return gram


def train_transform(image_size):
    """ Transforms for training images """
    transform = transforms.Compose(
        [
            transforms.Resize(int(image_size * 1.15)),
            transforms.RandomCrop(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean, std),
        ]
    )
    return transform


def style_transform(image_size=None):
    """ Transforms for style image """
    resize = [transforms.Resize(size=(image_size, image_size))] if image_size else []
    transform = transforms.Compose(resize + [transforms.ToTensor(), transforms.Normalize(mean, std)])
    return transform


def denormalize(tensors):
    """ Denormalizes image tensors using mean and std """
    for c in range(3):
        tensors[:, c].mul_(std[c]).add_(mean[c])
    return tensors


def deprocess(image_tensor):
    """ Denormalizes and rescales image tensor """
    image_tensor = denormalize(image_tensor)[0]
    image_tensor *= 255
    image_np = torch.clamp(image_tensor, 0, 255).cpu().numpy().astype(np.uint8)
    image_np = image_np.transpose(1, 2, 0)
    return image_np


In [ ]:
import argparse
import os

class Options():
    def __init__(self):
        self.parser = argparse.ArgumentParser(description="Parser for Fast-Neural-Style")
        self.parser.add_argument("--dataset_path", type=str, required=True, help="path to training dataset")
        self.parser.add_argument("--style_image", type=str, default="styles/mosaic.jpg", help="path to style image with .jpeg format")
        self.parser.add_argument("--epochs", type=int, default=1, help="number of training epochs")
        self.parser.add_argument("--batch_size", type=int, default=4, help="batch size for training")
        self.parser.add_argument("--image_size", type=int, default=256, help="size of training images")
        self.parser.add_argument("--style_size", type=int, help="size of style image")
        self.parser.add_argument("--lr", type=float, default=1e-3, help="Learning rate")
        self.parser.add_argument("--lambda_content", type=float, default=1e5, help="Weight for content loss")
        self.parser.add_argument("--lambda_style", type=float, default=1e10, help="Weight for style loss")
        self.parser.add_argument("--checkpoint_model", type=str, help="Optional path to checkpoint model")
        self.parser.add_argument("--checkpoint_interval", type=int, default=2000, help="Batches between saving model")
        self.parser.add_argument("--sample_interval", type=int, default=1000, help="Batches between saving image samples")

    def parse(self):
        return self.parser.parse_args()


In [ ]:
import sys,os,argparse
from IPython.display import HTML
CONFIG_FILE = '.config_ipynb'
if os.path.isfile(CONFIG_FILE):
    with open(CONFIG_FILE) as f:
        sys.argv = f.read().split()
else:
    sys.argv = ['train.py',
                '--dataset_path', TRAIN_DATASET,
                '--style_image', TRAIN_STYLE_PATH
                ]

args = Options().parse()

In [ ]:
import torch
from collections import namedtuple
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F


class VGG16(torch.nn.Module):
    def __init__(self, requires_grad=False):
        super(VGG16, self).__init__()
        vgg_pretrained_features = models.vgg16(pretrained=True).features
        self.slice1 = torch.nn.Sequential()
        self.slice2 = torch.nn.Sequential()
        self.slice3 = torch.nn.Sequential()
        self.slice4 = torch.nn.Sequential()
        for x in range(4):
            self.slice1.add_module(str(x), vgg_pretrained_features[x])
        for x in range(4, 9):
            self.slice2.add_module(str(x), vgg_pretrained_features[x])
        for x in range(9, 16):
            self.slice3.add_module(str(x), vgg_pretrained_features[x])
        for x in range(16, 23):
            self.slice4.add_module(str(x), vgg_pretrained_features[x])
        if not requires_grad:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, X):
        h = self.slice1(X)
        h_relu1_2 = h
        h = self.slice2(h)
        h_relu2_2 = h
        h = self.slice3(h)
        h_relu3_3 = h
        h = self.slice4(h)
        h_relu4_3 = h
        vgg_outputs = namedtuple("VggOutputs", ["relu1_2", "relu2_2", "relu3_3", "relu4_3"])
        out = vgg_outputs(h_relu1_2, h_relu2_2, h_relu3_3, h_relu4_3)
        return out


class TransformerNet(torch.nn.Module):
    def __init__(self):
        super(TransformerNet, self).__init__()
        self.model = nn.Sequential(
            ConvBlock(3, 32, kernel_size=9, stride=1),
            ConvBlock(32, 64, kernel_size=3, stride=2),
            ConvBlock(64, 128, kernel_size=3, stride=2),
            ResidualBlock(128),
            ResidualBlock(128),
            ResidualBlock(128),
            ResidualBlock(128),
            ResidualBlock(128),
            ConvBlock(128, 64, kernel_size=3, upsample=True),
            ConvBlock(64, 32, kernel_size=3, upsample=True),
            ConvBlock(32, 3, kernel_size=9, stride=1, normalize=False, relu=False),
        )

    def forward(self, x):
        return self.model(x)


class ResidualBlock(torch.nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            ConvBlock(channels, channels, kernel_size=3, stride=1, normalize=True, relu=True),
            ConvBlock(channels, channels, kernel_size=3, stride=1, normalize=True, relu=False),
        )

    def forward(self, x):
        return self.block(x) + x


class ConvBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, upsample=False, normalize=True, relu=True):
        super(ConvBlock, self).__init__()
        self.upsample = upsample
        self.block = nn.Sequential(
            nn.ReflectionPad2d(kernel_size // 2), nn.Conv2d(in_channels, out_channels, kernel_size, stride)
        )
        self.norm = nn.InstanceNorm2d(out_channels, affine=True) if normalize else None
        self.relu = relu

    def forward(self, x):
        if self.upsample:
            x = F.interpolate(x, scale_factor=2)
        x = self.block(x)
        if self.norm is not None:
            x = self.norm(x)
        if self.relu:
            x = F.relu(x)
        return x


In [94]:
from subprocess import Popen
from pathlib import Path
import requests

def resolve_chat_id():
    url = f'https://api.telegram.org/bot{TG_NOTIFY_BOT_TOKEN}/getUpdates'
    response = requests.get(url)
    res = response.json().get('result')
    if res is not None and len(res) > 0:
        try:
            res = res[0]['message']['from']['id']
        except:
            pass
    return res

def send_document(filename, token, chat_id, text):
    url = f'https://api.telegram.org/bot{token}/sendDocument'
    data = {'chat_id': chat_id, 'caption': text}
    with open(filename, 'rb') as f:
        files = {'document': f}
        response = requests.post(url, data=data, files=files)
        print(response.json())


def send_file_to_tg_by_subprocess(filename:str) -> None:
    import subprocess
    try:
        path = Path(filename)
        full_path = path.resolve(strict=True)
        subprocess.run(['telegram-send', '--file', full_path])
    except FileNotFoundError:
        print("file not exists")
        pass
        # doesn't exist


def send_file_to_tg(filename:str, text:str = None) -> None:
    try:
        path = Path(filename)
        full_path = path.resolve(strict=True)
        print(full_path)
        if text is None:
            p = Popen(['telegram-send', '-i', full_path]) # something long running
        else:
            p = Popen(['telegram-send', '-i', full_path, '--caption', text]) # something long running
    except FileNotFoundError:
        print("file not exists")
        pass
        # doesn't exist


def send_file_to_tg(filename:str, text:str = None) -> None:
    try:
        path = Path(filename)
        full_path = path.resolve(strict=True)
        print(full_path)
        if text is None:
            p = Popen(['telegram-send', '-f', full_path]) # something long running
        else:
            p = Popen(['telegram-send', '-f', full_path, '--caption', text]) # something long running
    except FileNotFoundError:
        print("file not exists")
        pass
        # doesn't exist

In [78]:
resolve_chat_id()

573811408

In [95]:
send_file_to_tg("./ts.conf", 'ho')

/home/src/notebooks/ts.conf


In [ ]:
import argparse
import os
import sys
import random
from PIL import Image
import numpy as np
import torch
import glob
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.utils import save_image

if __name__ == "__main__":
    style_name = args.style_image.split("/")[-1].split(".")[0]
    os.makedirs(f"images/outputs/{style_name}-training", exist_ok=True)
    os.makedirs(f"checkpoints", exist_ok=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Create dataloader for the training data
    train_dataset = datasets.ImageFolder(args.dataset_path, train_transform(args.image_size))
    dataloader = DataLoader(train_dataset, batch_size=args.batch_size)

    # Defines networks
    transformer = TransformerNet().to(device)
    vgg = VGG16(requires_grad=False).to(device)

    # Load checkpoint model if specified
    if args.checkpoint_model:
        transformer.load_state_dict(torch.load(args.checkpoint_model))

    # Define optimizer and loss
    optimizer = Adam(transformer.parameters(), args.lr)
    l2_loss = torch.nn.MSELoss().to(device)

    # Load style image
    style = style_transform(args.style_size)(Image.open(args.style_image))
    style = style.repeat(args.batch_size, 1, 1, 1).to(device)

    # Extract style features
    features_style = vgg(style)
    gram_style = [gram_matrix(y) for y in features_style]

    # Sample 8 images for visual evaluation of the model
    image_samples = []
    for path in random.sample(glob.glob(f"{args.dataset_path}/*/*.jpg"), 8):
        image_samples += [style_transform(args.image_size)(Image.open(path))]
    image_samples = torch.stack(image_samples)

    def save_sample(batches_done):
        """ Evaluates the model and saves image samples """
        transformer.eval()
        with torch.no_grad():
            output = transformer(image_samples.to(device))
        image_grid = denormalize(torch.cat((image_samples.cpu(), output.cpu()), 2))
        save_image(image_grid, f"images/outputs/{style_name}-training/{batches_done}.jpg", nrow=4)
        transformer.train()

    for epoch in range(args.epochs):
        epoch_metrics = {"content": [], "style": [], "total": []}
        for batch_i, (images, _) in enumerate(dataloader):
            optimizer.zero_grad()

            images_original = images.to(device)
            images_transformed = transformer(images_original)

            # Extract features
            features_original = vgg(images_original)
            features_transformed = vgg(images_transformed)

            # Compute content loss as MSE between features
            content_loss = args.lambda_content * l2_loss(features_transformed.relu2_2, features_original.relu2_2)

            # Compute style loss as MSE between gram matrices
            style_loss = 0
            for ft_y, gm_s in zip(features_transformed, gram_style):
                gm_y = gram_matrix(ft_y)
                style_loss += l2_loss(gm_y, gm_s[: images.size(0), :, :])
            style_loss *= args.lambda_style

            total_loss = content_loss + style_loss
            total_loss.backward()
            optimizer.step()

            epoch_metrics["content"] += [content_loss.item()]
            epoch_metrics["style"] += [style_loss.item()]
            epoch_metrics["total"] += [total_loss.item()]

            sys.stdout.write(
                "\r[Epoch %d/%d] [Batch %d/%d] [Content: %.2f (%.2f) Style: %.2f (%.2f) Total: %.2f (%.2f)]"
                % (
                    epoch + 1,
                    args.epochs,
                    batch_i * args.batch_size,
                    len(train_dataset),
                    content_loss.item(),
                    np.mean(epoch_metrics["content"]),
                    style_loss.item(),
                    np.mean(epoch_metrics["style"]),
                    total_loss.item(),
                    np.mean(epoch_metrics["total"]),
                )
            )

            batches_done = epoch * len(dataloader) + batch_i + 1
            if batches_done % args.sample_interval == 0:
                save_sample(batches_done)
                send_file_to_tg(checkpoint_filename, checkpoint_filename)

            if args.checkpoint_interval > 0 and batches_done % args.checkpoint_interval == 0 or :
                style_name = os.path.basename(args.style_image).split(".")[0]
                checkpoint_filename = f"checkpoints/{style_name}_{batches_done}.pth"
                torch.save(transformer.state_dict(), f"checkpoints/{style_name}_{batches_done}.pth")
                send_file_to_tg(checkpoint_filename, checkpoint_filename)


[Epoch 1/1] [Batch 10125/40504] [Content: 1150463.12 (1332310.14) Style: 319875.41 (926799.25) Total: 1470338.50 (2259109.39)]

In [ ]:
!ls -al ./dataset/train2014 | wc

   1436   12917  107579


In [ ]:
!ls -al ./dataset

total 96
drwxr-xr-x 3 root root  4096 Apr  8 01:53 .
drwxr-xr-x 1 root root  4096 Apr  8 01:29 ..
drwxr-xr-x 2 root root 90112 Apr  8 01:51 train2014


In [ ]:
!rm -rf ./dataset/train2014_2

In [ ]:
!mv ./dataset/train2014_2/COCO_train2014_00000022* ./dataset/train2014/

In [ ]:
torch.save(transformer.state_dict(), f"checkpoints/{'mosaic'}_{358}.pth")

In [ ]:
import torch
from torch.autograd import Variable
import argparse
import os
import tqdm
from torchvision.utils import save_image
from PIL import Image

os.makedirs("images/outputs", exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = style_transform()

# Define model and load model checkpoint
transformer = TransformerNet().to(device)
transformer.load_state_dict(torch.load('/content/checkpoints/mosaic_4000.pth'))
transformer.eval()

img_path = '/content/styles/picasso_selfport1907.jpg'

# Prepare input
image_tensor = Variable(transform(Image.open(img_path))).to(device)
image_tensor = image_tensor.unsqueeze(0)

# Stylize image
with torch.no_grad():
    stylized_image = denormalize(transformer(image_tensor)).cpu()

# Save image
fn = img_path.split("/")[-1]
save_image(stylized_image, f"images/outputs/stylized-{fn}")

In [ ]:
f"images/outputs/stylized-{fn}"

'images/outputs/stylized-the_scream.jpg'